# Input Parameters:
    1 Pulsar : J035-4510 Vela
    2 Date : 30/12/2023
    3 Time_IST : 3:10:00 -- MJD_input (we want the phase of pulsar)
    4 Location: GMRT

# Some helpful funtions 

In [1]:
from astropy.time import Time, TimeDelta
def isttoutc(ist_datetime,offset):
# Given date and time
    
    given_date_time = Time(ist_datetime, format='iso', scale='utc')

    offset = 19800.0 # 5 hours and 30 minutes
    # Time difference to subtract
    time_difference = TimeDelta(offset, format='sec')

    # Subtract the time difference
    utc_datetime = given_date_time - time_difference
    return utc_datetime

In [2]:
def calculate_phase_and_frequency(mjd, tmid, rphase, coeff1, coeff2, coeff3,f0):
    # Calculate the time difference in minutes
    dt = ((mjd - tmid) * 1440)

    # Calculate phase
    phase = rphase + dt * 60 * f0 + coeff1 + dt * coeff2 + dt**2 * coeff3
    
    # Calculate frequency in Hz
    freq = f0+ (1 / 60) * (coeff2 + 2 * dt * coeff3)

    return phase, freq,dt

Pulsar File

In [2]:
!cat J0835-4510.par #Velar Ephidermis 

PSRJ           J0835-4510
RAJ             08:35:20.5554429         1  0.00000479120352431585   
DECJ           -45:10:33.38894           1  0.00005350998986295085   
F0             11.188233045353667218     1  3.5161673561672063491e-13
F1             -1.5463446874179076243e-11 1  7.4410865651184824406e-21
F2             9.1268920254487347502e-22 1  7.0906746758831428687e-29
DM             67.844558268051564053     

PEPOCH         56364                       
POSEPOCH       56364                       
DMEPOCH        56364                       

TRACK -2

START          56625.539824453673024     
FINISH         58470.515823477110526     

TZRMJD         57400.488076953103693       
TZRFRQ         1368.6020510000000741    
TZRSITE        pks   
TRES           1.111        
EPHVER         2                           
NE_SW          9.9610000000000000001       
CLK            TT(TAI)
MODE 1
UNITS          TDB
TIMEEPH        FB90
DILATEFREQ     N
PLANET_SHAPIRO N
T2CMETHOD      TEMPO
CORR

# MJD(Time of Transit at observation) 
The Astrosupport from GMRT has been used to get the transit IST : http://gmrt.ncra.tifr.res.in/~astrosupp/obs_setup/rst.html


In [30]:
#MJD(Time of Transit at observation)
Time_ist = '2023-12-30 02:37:42'
Time_utc = isttoutc(Time_ist,offset=19800.0)

Transit_time = Time(Time_utc,scale='utc')

MJD_transit= Transit_time.mjd #get obstime in MJD 
print(f'MJD_transit:{MJD_transit}')
print(f'Transit_date_time_UTC:{Time_utc}')

MJD_transit:60307.88034722222
Transit_date_time_UTC:2023-12-29 21:07:42.000


In [28]:
#MJD time of observation 
Time_ist = '2023-12-30 03:10:00'
Time_utc = isttoutc(Time_ist,offset=19800.0)
observation_date = Time_utc
observation_time = Time(observation_date,scale='utc')

MJD_observation= observation_time.mjd #get obstime in MJD 
print(f'MJD_observation_UTC:{MJD_observation}')
print(f'observation_date_time_UTC:{observation_date}')


MJD_observation_UTC:60307.90277777778
observation_date_time_UTC:2023-12-29 21:40:00.000


In [31]:
difference_trans_obs_MJD_in_min = (MJD_observation-MJD_transit)*1440
print(f'{difference_trans_obs_MJD_in_min} min')  

32.300000006798655 min


In [32]:
#Tempo Command 
!tempo2 -f J0835-4510.par -polyco "60307.88034722222  60307.88034722222  10 5 1 GMRT 320" -tempo1

This program comes with ABSOLUTELY NO WARRANTY.
This is free software, and you are welcome to redistribute it
under conditions of GPL license.

************************************************* 
************************************************* 
[tempo2Util.C:396] Warning: [CLK3] no clock corrections available for clock UTC(gmrt) for MJD 60307.8
[tempo2Util.C:396] Warning: [CLK4] Trying assuming UTC = UTC(gmrt)
[tempo2Util.C:396] Warning: [CLK9] ... ok, using stated approximation 
[tempo2Util.C:401] Warning: [DUP1] duplicated warnings have been suppressed.
[tempo2Util.C:396] Warning: [CLK10] MJD is outside of UT1 table range  (MJD 60307.840269)
[tempo2Util.C:396] Warning: [CLK6] Proceeding assuming UTC =  UTC(gmrt)


In [33]:
!cat polyco_new.dat

0835-4510  29-Dec-23  200000.00   60307.83333333330            67.844624 -0.416 -7.668
   3811469661.982121   11.188233045354 GMRT   10    5   320.000                
  1.39108947790979706e-08 -2.85073211181204267e-01 -1.59025527755199667e-06
 -6.26260424464462072e-10  3.24520240589206457e-12
0835-4510  29-Dec-23  201000.00   60307.84027777770            67.844624 -0.415 -8.139
   3811476372.071014   11.188233045354 GMRT   10    5   320.000                
 -9.22872611686288725e-09 -2.85105198708113960e-01 -1.60792840718975499e-06
 -5.63808232389748896e-10 -7.58714873186081590e-14
0835-4510  29-Dec-23  202000.00   60307.84722222210            67.844624 -0.415 -7.780
   3811483082.159585   11.188233045354 GMRT   10    5   320.000                
 -1.05725466415063507e-09 -2.85137520797105248e-01 -1.62382201240711224e-06
 -4.40918411931255916e-10  6.08743162584846413e-12
0835-4510  29-Dec-23  203000.00   60307.85416666660            67.844624 -0.414 -8.185
   3811489792.247927   11.18823

# Relevant Output : 
  
  Please note that the current polyco output contained(2nd output) a MJD which was equal to the desired MJD at observation so we can compare the phase

  0835-4510  29-Dec-23  213000.00   60307.89583333320            67.844624 -0.411 -8.161
    3811530052.770683   11.188233045354 GMRT   10    5   320.000                
  -5.07435162082056869e-09 -2.85368268240268230e-01 -1.64980062997119053e-06
    2.05159184736838285e-10  5.62400736183134653e-12
    
    
  0835-4510  29-Dec-23  214000.00   60307.90277777770            67.844624 -0.411 -7.781
    3811536762.856716   11.188233045354 GMRT   10    5   320.000                
  -1.29613733244535179e-09 -2.85401183953524262e-01 -1.64106775768084792e-06
    3.26330968597264355e-10 -1.79881031323630087e-12

In [34]:
phase_at_MJD_observation = 3811536762.856716 #60307.90277777770 

phase_at_closest_MJD_to_observation = 3811530052.770683 #60307.89583333320 

f0 = 11.188233045354
rphase = phase_at_closest_MJD_to_observation
tmid = 60307.89583333320
coeff1 = -5.07435162082056869e-09
coeff2 = -2.85368268240268230e-01 
coeff3 = -1.64980062997119053e-06

phase,freq,dt = calculate_phase_and_frequency(MJD_observation,tmid,rphase,coeff1, coeff2, coeff3,f0)

print(f"Phase of J0835-4510 at {MJD_observation} at 320 MHz is {phase} which is interplotated from TMID:{tmid} having dt : {dt}")

Phase of J0835-4510 at 60307.90277777778 at 320 MHz is 3811536762.8567967 which is interplotated from TMID:60307.8958333332 having dt : 10.000000200234354


The Code blocks below this are in progress and no to be referred in the current update

0835-4510  13-Mar-13       0.00   56364.00000000000            67.844618  0.153 -8.519
         3343.026367   11.188233045354  pks   60    5  1368.602                
 -2.17721145166213902e-09 -1.02385926753534636e-02  1.10890314856534078e-06
 -3.43592390363715034e-10 -2.11309512519152620e-12

Finding the phase 0.0 MJD (It is away from TMID)

In [11]:
phase_at_MJD_observation = 3811536762.856716

phase_at_closest_MJD_to_observation =  3343.026367 

mjd_in = 56363.996541632165
f0 = 11.188233045354
rphase = phase_at_closest_MJD_to_observation
tmid = 56364.00000000000 
coeff1 = -2.17721145166213902e-09
coeff2 = -1.02385926753534636e-02 
coeff3 = 1.10890314856534078e-06

phase,freq,dt = calculate_phase_and_frequency(mjd_in,tmid,rphase,coeff1, coeff2, coeff3,f0)

print(f"Phase of J0835-4510 at {mjd_in} is {phase} which is interplotated from TMID:{tmid} having dt : {dt}")

Phase of J0835-4510 at 56363.996541632165 is -2.5039543839376285e-06 which is interplotated from TMID:56364.0 having dt : -4.9800496827811


In [12]:
-2.5039543839376285e-06/11.188233045354

-2.2380248729064639e-07

0900-3144  17-Jun-09       0.00   54999.99999999990            62.585897  0.619 -7.088
        -5455.503312   90.011841925172 AXIS   60    5  1440.000 0.2688   0.0534
  7.93209302820642315e-08 -6.09967098486254189e-01 -1.60419608060829332e-05
  1.95943255712137684e-08 -1.61621252490747432e-11

In [27]:
phase_at_closest_MJD_to_observation =  318550.476117 

mjd_in = 52715.467459566835657
f0 = 90.011841925172
rphase = phase_at_closest_MJD_to_observation
tmid = 54999.99999999990  
coeff1 = 7.93209302820642315e-08
coeff2 = -6.09967098486254189e-01 
coeff3 = -1.60419608060829332e-05

phase,freq,dt = calculate_phase_and_frequency(mjd_in,tmid,rphase,coeff1, coeff2, coeff3,f0)

print(f"Phase of J0835-4510 at {mjd_in} is {phase} which is interplotated from TMID:{tmid} having dt : {dt}")

Phase of J0835-4510 at 52715.46745956683 is -17938148218.11908 which is interplotated from TMID:54999.9999999999 having dt : -3289726.858223614


In [26]:

phase_at_closest_MJD_to_observation =  318550.476117 

mjd_in = 54999.95903491644
f0 = 90.011841925172
rphase = phase_at_closest_MJD_to_observation
tmid = 54999.99999999990  
coeff1 = 7.93209302820642315e-08
coeff2 = -6.09967098486254189e-01 
coeff3 = -1.60419608060829332e-05

phase,freq,dt = calculate_phase_and_frequency(mjd_in,tmid,rphase,coeff1, coeff2, coeff3,f0)

print(f"Phase of J0835-4510 at {mjd_in} is {phase} which is interplotated from TMID:{tmid} having dt : {dt}")

Phase of J0835-4510 at 54999.95903491644 is 2.2790350008135285e-05 which is interplotated from TMID:54999.9999999999 having dt : -58.98972017574124


In [24]:
0.004039893627631484/90.011841925172

4.488180156328652e-05

In [25]:
54999.95903491696- (4.488180156328652e-05/(60*1440))

54999.95903491644

In [10]:
56363.996541684945-(0.004559880991057786/(60*1440))

56363.996541632165

Trial for prediction of desired MJD in polyco output (more tests needed)

In [37]:
mjd1_gmrt=60307.697421938006 #desired output
mjd1_gmrt_frac = round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
mjd1_gmrt_frac_new = round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
ntmp1 = int(mjd1_gmrt_frac * 1.0e8)
ntmp2 = int((mjd1_gmrt_frac * 1.0e8 - ntmp1) * 100.0)
mjd1 = ntmp2 * 1e-10 + ntmp1 * 1.0e-8
mjd1_gmrt_frac=mjd1
mjd1_gmrt_round = int(mjd1_gmrt) + mjd1_gmrt_frac#round(48.0 * (mjd1_gmrt - int(mjd1_gmrt))) / 48.0
mjd1_gmrt_round_new = int(mjd1_gmrt) + mjd1_gmrt_frac_new
print(mjd1_gmrt_round_new)



60307.6875


In [24]:
(60307.90277777778-60307.89583333333)*1440 #difference in min

10.000000011641532

In [35]:
!tempo2 -f J0835-4510.par -polyco "60307.697421938006 60307.697421938006 10 5 2 GMRT 320" -tempo1

This program comes with ABSOLUTELY NO WARRANTY.
This is free software, and you are welcome to redistribute it
under conditions of GPL license.

************************************************* 
************************************************* 
[tempo2Util.C:396] Warning: [CLK3] no clock corrections available for clock UTC(gmrt) for MJD 60307.8
[tempo2Util.C:396] Warning: [CLK4] Trying assuming UTC = UTC(gmrt)
[tempo2Util.C:396] Warning: [CLK9] ... ok, using stated approximation 
[tempo2Util.C:401] Warning: [DUP1] duplicated warnings have been suppressed.
[tempo2Util.C:396] Warning: [CLK10] MJD is outside of UT1 table range  (MJD 60307.798602)
[tempo2Util.C:396] Warning: [CLK6] Proceeding assuming UTC =  UTC(gmrt)


In [36]:
!cat polyco_new.dat

0835-4510  29-Dec-23  190000.00   60307.79166666660            67.844624 -0.418 -7.667
   3811429401.441963   11.188233045354 GMRT   10    5   320.000                
  1.84862641574367589e-08 -2.84891323516309691e-01 -1.42499445274617452e-06
 -1.15501346281816550e-09 -2.09925360119294270e-12
0835-4510  29-Dec-23  191000.00   60307.79861111100            67.844624 -0.418 -8.085
   3811436111.532690   11.188233045354 GMRT   10    5   320.000                
 -1.53351931178803808e-09 -2.84920176092655020e-01 -1.45893548262236686e-06
 -1.09795154231774865e-09 -1.91281364778666167e-12
0835-4510  29-Dec-23  192000.00   60307.80555555540            67.844624 -0.418 -7.770
   3811442821.623125   11.188233045354 GMRT   10    5   320.000                
 -6.27557149185845511e-09 -2.84949683525734077e-01 -1.49142811803467622e-06
 -9.92348155189048760e-10  7.29763834014796089e-12
0835-4510  29-Dec-23  193000.00   60307.81249999990            67.844624 -0.417 -8.084
   3811449531.713359   11.18823

Relevant Output : 
0835-4510  29-Dec-23  214000.00   60307.90277777770            67.844624 -0.411 -8.133
   3811536762.856716   11.188233045354 GMRT   10    5   320.000                
 -9.72918089698028823e-09 -2.85401182490145344e-01 -1.64137787605378730e-06
  2.95211622107113556e-10  1.13667726003359661e-12
